# Building the CASBAH Galaxy Database (v1.0)

## SDSS

### Targeting
I am unclear on how to sensibly extract targeting information from the
SDSS.  But this may well be an issue for various analyses.

### Extracting Galaxy data
The script build_sdss loops through the listed fields with SDSS
coverage and calls the grab_sdss_spectra script to grab photometric
and spectral data.  It is currently grabbing DR12.

Here are some defaults:
* Box of 2deg on a side
* Photometry is Petrosian ugriz
* Galaxies are cut down to 20Mpc separation (LCDM cosmology)
* z > 500km/s to cut stars

Here is the basic procedure:
* Query photometry all objects in search box with spectra
* Query list of spectra from SDSS in search box
    * This list often contain duplicates from multiple passes
* Cut on 20Mpc using redshifts, RA+DEC of obj, and RA+DEC of QSO
* Loop on sources to build table
    * Take BOSS data over SDSS
* Generate a binary FITS table, including photometry, redshift and spectra

In [5]:
# Example call
from xastropy.casbah import galaxy_data as xcgd
reload(xcgd)
radec = (212.34957*u.deg,26.30585*u.deg)
tab=xcgd.grab_sdss_spectra(radec, radius=1.*u.degree/12.) 
Table(tab[0:5])

grab_sdss_spectra: Found 12 sources in the search box.
grab_sdss_spectra: Grabbing data for 12 sources.
Wrote SDSS table to None


RA,DEC,Z,Z_ERR,SDSS_MAG [5],SDSS_MAGERR [5],TELESCOPE,INSTRUMENT,FLUX [5000],SIG [5000],WAVE [5000]
float64,float64,float64,float64,float64,float64,string640,string640,float32,float32,float64
212.35102636,26.3415403908,3.21771526337,0.000997702358291,23.70004 .. 7.34031909445e+223,1.576056 .. 6.013470017e-154,SDSS 2.5-M,BOSS,3.5592 .. 1.35632e-19,2.65591 .. 1.35632e-19,3592.52587891 .. -1.22810024928e-296
212.349627859,26.3058613203,0.0,0.0,16.10909 .. -6.11925950261e-220,0.004779854 .. -2.78592920103e-305,SDSS 2.5-M,SDSS,0.0 .. 1.77053e-38,0.0 .. -1.21568e-37,3811.53564453 .. 1.57733159326e-52
212.370094746,26.3254555654,0.327256381512,5.58089996048e-05,21.36104 .. 6.013470017e-154,1.245788 .. 1.83037365249e-94,SDSS 2.5-M,BOSS,3.59836 .. -1.19132e+14,3.71348 .. -9.48227e+21,3590.87353516 .. 8.31176848954e+46
212.363198097,26.3766678271,0.673991203308,0.000221190101001,21.94763 .. 1.76807794921e+258,1.772229 .. -1.47312822905e-204,SDSS 2.5-M,BOSS,-0.0600585 .. 1.35632e-19,2.7654 .. -1.47499e+12,3592.52587891 .. 7.86200159392e-51
212.272722288,26.305233155,0.321305960417,8.39712593006e-05,21.33263 .. 3.00282156321e-142,0.844611 .. -1.19464315864e-195,SDSS 2.5-M,BOSS,-0.931015 .. 168.798,2.87149 .. 3.81592e+09,3590.87353516 .. -1.10999478584e-51


### Open questions/issues
1.  Should we be recording other observing meta-data?
1.  How about SFR, M*, etc.?

## DEIMOS

### Targeting

* Pull mask target info from Mask files :: parse_deimos_mask_file
* Pull other target info from SExtractor output
  * Requires yaml file describing target criteria
  * And the SExtractor output file

#### Sample output of MULTI_OBS file

MULTI_OBJ file:

|  INSTR |        MASK_NAME |     MASK_RA |   MASK_DEC | MASK_EPOCH | MASK_PA |    DATE_OBS | DISPERSER |   TEXP |         CONDITIONS |

| DEIMOS | PG1407_may_early | 14:09:34.10 | 26:18:45.1 |     2000.0 |   -96.1 | 23-Jul-2015 |      G600 | 3600.0 | POOR_SEEING,CLOUDS |

| DEIMOS | PG1407_may_early | 14:09:34.10 | 26:18:45.1 |     2000.0 |   -96.1 | 24-Jul-2015 |      G600 | 3600.0 |              CLEAR |


In [75]:
#### Sample of target file
fil='/Users/xavier/CASBAH/Galaxies/PG1407+265/PG1407+265_targets.fits'
targ = Table.read(fil)
#
mt = np.where(targ['MASK_NAME'] != 'N/A')[0]
targ[mt[0:5]]

TARG_RA,TARG_DEC,EPOCH,TARG_ID,TARG_MAG,TARG_IMG,INSTR,MASK_NAME
float64,float64,float64,int32,float32,string136,string192,string512
212.39539331,26.1409489801,2000.0,556,23.7325,pg1407.V.mos.fits,DEIMOS,PG1407_may_mid2
212.386193606,26.1464415539,2000.0,703,23.8043,pg1407.V.mos.fits,DEIMOS,PG1407_may_mid2
212.365282515,26.149680065,2000.0,742,20.7288,pg1407.V.mos.fits,DEIMOS,PG1407_may_mid2
212.392594184,26.1567769971,2000.0,901,19.9971,pg1407.V.mos.fits,DEIMOS,PG1407_may_mid2
212.390583821,26.1596075882,2000.0,1037,24.3026,pg1407.V.mos.fits,DEIMOS,PG1407_may_mid2


## Testing

In [77]:
fil='/Users/xavier/CASBAH/Galaxies/PG1407+265/PG1407+265_targets.fits'
tmp = Table.read(fil,fill_values=[('N/A','0','MASK_NAME')],format='fits')

TypeError: read_table_fits() got an unexpected keyword argument 'fill_values'